<a href="https://colab.research.google.com/github/andreidm92/Agents_in_code/blob/main/practice/Lesson_14_sql_bot_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 14 — Data Scientist: Nursing Protocol Finder
### 🎯 Цель: создать ассистента для Data Scientist, который находит протоколы лечения по диагнозам в SQL-базе

📌 **Profession = Data Scientist** → ты разрабатываешь агента-помощника для аналитика данных в медицине

**SQLDatabaseChain** интерпретирует вопросы на естественном языке и выполняет SQL-запросы
**Graph Versioning** позволяет сохранять и переключаться между версиями графов агентов (для A/B тестов, отладки и т.п.)

In [13]:
# Установка зависимостей
!pip install -q langchain langchain-community langchain-experimental openai ipywidgets pandas
!pip install -q -U langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 23.1 MB/s eta 0:00:00


In [14]:
# Импорты
from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain_openai import OpenAI
import pandas as pd
import sqlite3
from IPython.display import display
import ipywidgets as widgets

In [7]:
import os, getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("Вставь OpenAI API ключ: ")

Вставь OpenAI API ключ: ··········


In [11]:
# Создание расширенной базы protocols.db с большим числом строк
conn = sqlite3.connect("protocols.db")
#Курсор — это как “интерпретатор SQL внутри Python”.
cursor = conn.cursor()

# Удаляем таблицу, если она была
cursor.execute("DROP TABLE IF EXISTS nursing_protocols")

# Создаём таблицу
cursor.execute("""
CREATE TABLE nursing_protocols (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    diagnosis TEXT,
    treatment TEXT,
    dosage TEXT,
    notes TEXT
)
""")

# Добавляем 12 диагнозов с разными данными
data = [
    ("Анемия", "Железосодержащие препараты", "100 мг 2 раза в день", "Контроль уровня ферритина"),
    ("Гипертония", "Ингибиторы АПФ", "5 мг в сутки", "Регулярное измерение давления"),
    ("Диабет 2 типа", "Метформин", "500 мг 2 раза в день", "Глюкоза до и после еды"),
    ("Остеопороз", "Кальций и витамин D", "Кальций 1000 мг, D3 800 МЕ", "Уровень кальция в крови"),
    ("Астма", "Ингаляционные кортикостероиды", "2 вдоха утром и вечером", "Контроль пикфлоуметрии"),
    ("Гастрит", "Ингибиторы протонной помпы", "20 мг в сутки", "Перед завтраком 30 мин"),
    ("Пневмония", "Амоксициллин", "500 мг 3 раза в день", "Минимум 7 дней при лихорадке"),
    ("ОРВИ", "Симптоматическая терапия", "Парацетамол по необходимости", "Обильное питьё и покой"),
    ("Мигрень", "Суматриптан", "50 мг при приступе", "Макс 200 мг в сутки"),
    ("Инсулинорезистентность", "Физнагрузка + метформин", "500–850 мг 2 раза в день", "Диета с низким ГИ"),
    ("Цистит", "Нитрофурантоин", "100 мг 2–3 раза в день", "Курс 5–7 дней"),
    ("Артрит", "НПВС", "Ибупрофен 400 мг по мере необходимости", "Не более 3 раз в день")
]

cursor.executemany("INSERT INTO nursing_protocols (diagnosis, treatment, dosage, notes) VALUES (?, ?, ?, ?)", data)
conn.commit()
conn.close()

# Проверим
df_check = pd.read_sql("SELECT * FROM nursing_protocols", sqlite3.connect("protocols.db"))
df_check


,id,diagnosis,treatment,dosage,notes
0,1,Анемия,Железосодержащие препараты,100 мг 2 раза в день,Контроль уровня ферритина
1,2,Гипертония,Ингибиторы АПФ,5 мг в сутки,Регулярное измерение давления
2,3,Диабет 2 типа,Метформин,500 мг 2 раза в день,Глюкоза до и после еды
3,4,Остеопороз,Кальций и витамин D,"Кальций 1000 мг, D3 800 МЕ",Уровень кальция в крови
4,5,Астма,Ингаляционные кортикостероиды,2 вдоха утром и вечером,Контроль пикфлоуметрии
5,6,Гастрит,Ингибиторы протонной помпы,20 мг в сутки,Перед завтраком 30 мин
6,7,Пневмония,Амоксициллин,500 мг 3 раза в день,Минимум 7 дней при лихорадке
7,8,ОРВИ,Симптоматическая терапия,Парацетамол по необходимости,Обильное питьё и покой
8,9,Мигрень,Суматриптан,50 мг при приступе,Макс 200 мг в сутки
9,10,Инсулинорезистентность,Физнагрузка + метформин,500–850 мг 2 раза в день,Диета с низким ГИ


In [15]:
# Инициализация SQLDatabaseChain
db = SQLDatabase.from_uri("sqlite:///protocols.db")
llm = OpenAI(temperature=0)
chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)


🔍 Что делает:

chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

Создаёт SQL-агента, который:

- Принимает вопрос на естественном языке;
- Использует LLM, чтобы сгенерировать SQL-запрос;
- Отправляет этот SQL-запрос в db;
- Возвращает результат пользователю.

⚙️ verbose=True — включает печать промежуточных SQL-запросов (полезно для отладки).

Text(value='', description='Вопрос:', layout=Layout(width='100%'), placeholder='Например: Что назначают при ан…

Button(description='Отправить', style=ButtonStyle())

Output()

## 🧪 Демонстрация Graph Versioning (LangGraph)
**Пояснение:** сохранение и переключение версий графа позволяет безопасно тестировать изменения

*Пример — подграф с условием: если вопрос содержит 'протокол', вызываем SQL, иначе просто LLM* (не реализовано полностью, псевдокод):

In [18]:
!pip install -q -U langgraph


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 16.4 MB/s eta 0:00:00


In [21]:
# Генерация актуального примера: StateGraph + ручное "сохранение версий"
corrected_graph_example = '''# ✅ Актуальный пример LangGraph: компиляция и эмуляция версий

!pip install -U langgraph

from langgraph.graph import StateGraph
from typing import TypedDict
import json

# 1. Определим схему состояния
class State(TypedDict):
    step: int
    log: bool

# 2. Версия V1
graph_v1 = StateGraph(State)
graph_v1.add_node("start", lambda state: {"step": 1})
graph_v1.set_entry_point("start")

compiled_v1 = graph_v1.compile()
graph_dict_v1 = graph_v1.get_graph()  # сериализуем как словарь

with open("graph_v1.json", "w") as f:
    json.dump(graph_dict_v1, f, indent=2)

# 3. Версия V2 (добавим лог)
graph_v2 = StateGraph(State)
graph_v2.add_node("start", lambda state: {"step": 1})
graph_v2.add_node("log", lambda state: {"log": True})
graph_v2.add_edge("start", "log")
graph_v2.set_entry_point("start")

compiled_v2 = graph_v2.compile()
graph_dict_v2 = graph_v2.get_graph()

with open("graph_v2.json", "w") as f:
    json.dump(graph_dict_v2, f, indent=2)

# 4. Сравнение (на уровне структуры)
print("🧠 V1 nodes:", list(graph_dict_v1["nodes"].keys()))
print("🧠 V2 nodes:", list(graph_dict_v2["nodes"].keys()))
'''

corrected_graph_example


'# ✅ Актуальный пример LangGraph: компиляция и эмуляция версий\n\n!pip install -U langgraph\n\nfrom langgraph.graph import StateGraph\nfrom typing import TypedDict\nimport json\n\n# 1. Определим схему состояния\nclass State(TypedDict):\n    step: int\n    log: bool\n\n# 2. Версия V1\ngraph_v1 = StateGraph(State)\ngraph_v1.add_node("start", lambda state: {"step": 1})\ngraph_v1.set_entry_point("start")\n\ncompiled_v1 = graph_v1.compile()\ngraph_dict_v1 = graph_v1.get_graph()  # сериализуем как словарь\n\nwith open("graph_v1.json", "w") as f:\n    json.dump(graph_dict_v1, f, indent=2)\n\n# 3. Версия V2 (добавим лог)\ngraph_v2 = StateGraph(State)\ngraph_v2.add_node("start", lambda state: {"step": 1})\ngraph_v2.add_node("log", lambda state: {"log": True})\ngraph_v2.add_edge("start", "log")\ngraph_v2.set_entry_point("start")\n\ncompiled_v2 = graph_v2.compile()\ngraph_dict_v2 = graph_v2.get_graph()\n\nwith open("graph_v2.json", "w") as f:\n    json.dump(graph_dict_v2, f, indent=2)\n\n# 4. Сра

In [30]:
# Импорты
from langgraph.graph import StateGraph
from typing import TypedDict

# Описание состояния
class State(TypedDict):
    step: int
    log: bool

# 🎯 Граф V1
graph_v1 = StateGraph(State)
graph_v1.add_node("start", lambda state: {"step": 1})
graph_v1.set_entry_point("start")
compiled_v1 = graph_v1.compile()

# 🎯 Граф V2: добавим шаг с логированием
graph_v2 = StateGraph(State)
graph_v2.add_node("start", lambda state: {"step": 1})
graph_v2.add_node("log_step", lambda state: {"log": True})  # ✅ безопасное имя
graph_v2.add_edge("start", "log_step")
graph_v2.set_entry_point("start")
compiled_v2 = graph_v2.compile()

# 📊 Сравнение графов
print("📌 V1 nodes:", list(compiled_v1.get_graph().nodes.keys()))
print("📌 V2 nodes:", list(compiled_v2.get_graph().nodes.keys()))
print("📌 V1 edges:", compiled_v1.get_graph().edges)
print("📌 V2 edges:", compiled_v2.get_graph().edges)


📌 V1 nodes: ['__start__', 'start', '__end__']
📌 V2 nodes: ['__start__', 'start', 'log_step', '__end__']
📌 V1 edges: [Edge(source='__start__', target='start', data=None, conditional=False), Edge(source='start', target='__end__', data=None, conditional=False)]
📌 V2 edges: [Edge(source='__start__', target='start', data=None, conditional=False), Edge(source='start', target='log_step', data=None, conditional=False), Edge(source='log_step', target='__end__', data=None, conditional=False)]
